# Euler-Bernoulli beam

We discuss the solutions of the Euler-Bernoulli beam in case of a uniformly distributed body force and a shear force at the end. It is clamped on the right side. Ignoring material parameters, the Euler-Bernoulli equation for a given body force is the fourth-order problem
\begin{align*}
\frac{1}{12}w^{\prime\prime\prime\prime} = f,\qquad + \text{b.c.},
\end{align*}
where $w$ is the vertical deflection. By integration the exact solution of the Euler-Bernoulli beam is of the form
\begin{align*}
w(x) = \int\int\int\int \frac{f}{12}\,dx\,dx\,dx\,dx +\frac{c_1}{6}x^3+\frac{c_2}{2}x^2+c_3x+c_4 = \frac{q}{2}x^4+\frac{c_1}{6}x^3+\frac{c_2}{2}x^2+c_3x+c_4
\end{align*}
for $f=q=\mathrm{const}$. The constants are determined by the boundary conditions. For a clamped boundary we have the conditions
\begin{align*}
w=w^{\prime}=0.
\end{align*}
For a free-boundary we have
\begin{align*}
w^{\prime\prime}=w^{\prime\prime\prime}=0.
\end{align*}
Note that $\sigma=w^{\prime\prime}$ is the bending moment and with $\frac{1}{12} w^{\prime\prime\prime}=q$ a shear force can be implied. Therefore, in case of a shear force we have $f=0$ as body force and thus,
\begin{align*}
w(x) =\frac{c_1}{6}x^3+\frac{c_2}{2}x^2+c_3x+c_4
\end{align*}
and $\frac{1}{12} w^{\prime\prime\prime}=q$ as a boundary condition.

In [ ]:
from ngsolve import *
from ngsolve.meshes import Make1DMesh
from ngsolve.webgui import Draw
from sympy import symbols, solve, diff, lambdify

# use material parameters such that equations simplify
mu, lam = 0.5, 0  # Lame parameter
nu = lam/(2*(lam+mu)) # Possion ratio (=0)
E = mu*(3*lam+2*mu)/(lam+mu) # Young's modulus (=1)

## Uniform distributed load

First, we consider a uniformly distributed body force (gravity).

In [ ]:
# uniform distributed load
q = -0.1

X, C1, C2, C3, C4 = symbols("x C1 C2 C3 C4")
w_ex = q/2*X**4+C1/6*X**3+C2/2*X**2+C3*X+C4
w_diff_ex = 2*q*X**3+C1/2*X**2+C2*X+C3
w_ddiff_ex = 6*q*X**2+C1*X+C2


# solve for boundary conditions
solve_bc = solve((w_ex.subs(X,0), w_diff_ex.subs(X,0), w_ddiff_ex.subs(X,1), \
                diff(w_ddiff_ex,X).subs(X,1)), C1, C2, C3, C4)
c1,c2,c3,c4 = solve_bc[C1], solve_bc[C2], solve_bc[C3], solve_bc[C4]

# generate CoefficientFunctions
w_ex_func = lambdify((X), w_ex.subs([(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
cf_w_ex = w_ex_func(x)
cf_sigma_ex = 1/12*cf_w_ex.Diff(x).Diff(x)


# Test boundary conditions
print("test: w(0)    = ", w_ex.subs([(X,0),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
print("test: w'(0)   = ", w_diff_ex.subs([(X,0),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
print("test: w''(1)  = ", w_ddiff_ex.subs([(X,1),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
print("test: w'''(1) = ", diff(w_ddiff_ex,X).subs([(X,1),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))

print("test: w(1)    = ", w_ex.subs([(X,1),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))

Draw the exact deflection and moment.

In [ ]:
mesh = Make1DMesh(10)
Draw(cf_w_ex, mesh, deformation=CF( (0, cf_w_ex, 0) ))
Draw(cf_sigma_ex, mesh, deformation=CF( (0, cf_sigma_ex, 0) ))

A procedure to solve the Euler-Bernoulli beam using a mixed method using the vertical deflection and the bending moment as unknowns. We can use Lagrange finite elements instead of $C^1$-conforming elements (such as Hermite polynomials).

In [ ]:
def SolveBB(order, mesh, draw=True):
    fes = H1(mesh, order=order, dirichlet="left")*H1(mesh, order=order, dirichlet="right")
    (u,sigma),(du,dsigma) = fes.TnT()
    
    a = BilinearForm(fes, symmetric=True)
    a += (12*sigma*dsigma + grad(u)*grad(dsigma) + grad(du)*grad(sigma) -1e-10*u*du)*dx
    
    f = LinearForm(fes)
    f += -q*du*dx
    
    gfsol = GridFunction(fes)
    
    a.Assemble()
    f.Assemble()
    gfsol.vec.data = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")*f.vec
    gfw, gfsigma = gfsol.components

    if draw:
        Draw(gfw, mesh, deformation=CF((0, gfw, 0)))
        Draw(gfsigma, mesh, deformation=CF((0, gfsigma, 0)))

    # relative errors
    errW = sqrt(Integrate((gfw-cf_w_ex)**2+(Grad(gfw)-cf_w_ex.Diff(x))**2,mesh))/sqrt(Integrate(cf_w_ex**2+cf_w_ex.Diff(x)**2,mesh))
    errsigma = sqrt(Integrate((gfsigma-cf_sigma_ex)**2+(Grad(gfsigma)-cf_sigma_ex.Diff(x))**2,mesh))/sqrt(Integrate(cf_sigma_ex**2+cf_sigma_ex.Diff(x)**2,mesh))
    
    return (gfw(mesh(1,0,0)), errW, errsigma)

tip_deflection, errW, errSigma = SolveBB(order=1, mesh=Make1DMesh(10), draw=True)
print(f"{tip_deflection=}, {errW=}, {errSigma=}")

## Shear force
Next, we apply a shear force at the right end.

In [ ]:
# shear force
q = -0.1


X, C1, C2, C3, C4 = symbols("x C1 C2 C3 C4")
w_ex = C1/6*X**3+C2/2*X**2+C3*X+C4
w_diff_ex = C1/2*X**2+C2*X+C3
w_ddiff_ex = C1*X+C2


# solve for boundary conditions
solve_bc = solve((w_ex.subs(X,0), w_diff_ex.subs(X,0), \
                  w_ddiff_ex.subs(X,1), 1/12*diff(w_ddiff_ex,X).subs(X,1)+q), C1, C2, C3, C4)
c1,c2,c3,c4 = solve_bc[C1], solve_bc[C2], solve_bc[C3], solve_bc[C4]

# generate CoefficientFunctions
w_ex_func = lambdify((X), w_ex.subs([(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
cf_w_ex = w_ex_func(x)
cf_sigma_ex = 1/12*cf_w_ex.Diff(x).Diff(x)


# Test boundary conditions
print("test: w(0)    = ", w_ex.subs([(X,0),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
print("test: w'(0)   = ", w_diff_ex.subs([(X,0),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
print("test: w''(1)  = ", w_ddiff_ex.subs([(X,1),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
print("test: w'''(1) = ", diff(w_ddiff_ex,X).subs([(X,1),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))

print("test: w(1)    = ", w_ex.subs([(X,1),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))

mesh = Make1DMesh(10)
Draw(cf_w_ex,mesh, deformation=CF( (0, cf_w_ex, 0) ))
Draw(cf_sigma_ex,mesh, deformation=CF( (0, cf_sigma_ex, 0) ))

And solve it with the mixed FEM.

In [ ]:
def SolveBB(order, mesh, draw=True):
    fes = H1(mesh, order=order, dirichlet="left")*H1(mesh, order=order, dirichlet="right")
    (u,sigma),(du,dsigma) = fes.TnT()
    
    a = BilinearForm(fes, symmetric=True)
    a += (12*sigma*dsigma + grad(u)*grad(dsigma) + grad(du)*grad(sigma) -1e-10*u*du)*dx
    
    f = LinearForm(fes)
    f += -q*du*ds("right")
    
    gfsol = GridFunction(fes)
    
    a.Assemble()
    f.Assemble()
    gfsol.vec.data = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")*f.vec
    gfw, gfsigma = gfsol.components
    
    if draw:
        Draw(gfw, mesh, deformation=CF((0, gfw, 0)))
        Draw(gfsigma, mesh, deformation=CF((0, gfsigma, 0)))

    # relative errors
    errW = sqrt(Integrate((gfw-cf_w_ex)**2+(Grad(gfw)-cf_w_ex.Diff(x))**2,mesh))/sqrt(Integrate(cf_w_ex**2+cf_w_ex.Diff(x)**2,mesh))
    errsigma = sqrt(Integrate((gfsigma-cf_sigma_ex)**2+(Grad(gfsigma)-cf_sigma_ex.Diff(x))**2,mesh))/sqrt(Integrate(cf_sigma_ex**2+cf_sigma_ex.Diff(x)**2,mesh))

    return (gfw(mesh(1,0,0)), errW, errsigma)

tip_deflection, errW, errSigma = SolveBB(order=1, mesh=Make1DMesh(10), draw=True)
print(f"{tip_deflection=}, {errW=}, {errSigma=}")